In [1]:
import json
from functools import partial
from pathlib import Path
from typing import Annotated, Optional
from uuid import uuid4

import anthropic
import instructor
import openai
from dotenv import load_dotenv
from pydantic import (
    AfterValidator,
    BaseModel,
    BeforeValidator,
    Field,
    ValidationInfo,
    create_model,
    field_validator,
    model_validator,
)
from tenacity import Retrying, RetryError, stop_after_attempt, wait_random_exponential

from dreamai.ai import (
    ModelName,
    assistant_message,
    count_gpt_tokens,
    system_message,
    user_message,
)
from dreamai.utils import deindent

load_dotenv()

ask_oai = instructor.from_openai(openai.OpenAI())
ask_cld = instructor.from_anthropic(anthropic.Anthropic())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
MODEL = ModelName.HAIKU
CONCEPT_WORD_COUNT = 3

In [96]:
def validate_word_count(text: str, word_count: int = 3, text_name: str = "Text") -> str:
    if len(text.split()) < word_count:
        raise ValueError(f"{text_name} should be at least {word_count} words long")
    return text


class Question(BaseModel):
    id: str
    problem: str
    solution: str


class TopicSubtopic(BaseModel):
    topic: str
    subtopic: str

    @model_validator(mode="after")  # type: ignore
    def validate_topic_subtopic(self, info: ValidationInfo) -> "TopicSubtopic":
        topics = info.context.get("topics")  # type: ignore
        topics_names = [topic.name for topic in topics]
        if self.topic not in topics_names:
            raise ValueError(f"Topic {self.topic} not found in topics")
        topic = next(topic for topic in topics if topic.name == self.topic)
        if self.subtopic not in [subtopic.name for subtopic in topic.subtopics]:
            raise ValueError(
                f"Subtopic {self.subtopic} not found in Topic: {self.topic}"
            )
        return self


class QuestionWithTopicSubtopic(Question):
    topic: str
    subtopic: str


class QuestionsWithTopicSubtopic(BaseModel):
    questions: list[QuestionWithTopicSubtopic]


class QuestionWithTopicSubtopicAndSubquestions(QuestionWithTopicSubtopic):
    subquestions: list[QuestionWithTopicSubtopic]


class Concept(BaseModel):
    concept: Annotated[
        str,
        AfterValidator(
            partial(
                validate_word_count, word_count=CONCEPT_WORD_COUNT, text_name="Concept"
            )
        ),
    ]


class QuestionWithConcept(QuestionWithTopicSubtopic):
    concept: str


class QuestionWithTopicSubtopicAndConceptSubquestions(QuestionWithTopicSubtopic):
    subquestions: list[QuestionWithConcept]


class QuestionWithConceptAndSubquestions(QuestionWithConcept):
    subquestions: list[QuestionWithConcept]


class Subtopic(BaseModel):
    name: str
    concepts: list[str] = Field(default_factory=list)


class Topic(BaseModel):
    name: str
    subtopics: list[Subtopic]

In [72]:
data_dir = Path("/media/hamza/data2/MATH/train/")

question_files = [
    file
    for file in data_dir.glob("**/*.json")
    if file.parent.name != "counting_and_probability"
]
questions = [json.loads(file.read_text()) for file in question_files]
questions = [
    Question(
        **{
            "id": str(i),
            "problem": question["problem"],
            "solution": question["solution"],
        }
    )
    for i, question in enumerate(questions)
    if "5" in question["level"]
]
questions = [q for q in questions if q.id in ["225", "6349"]]
questions[1].model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$'}

In [92]:
topics = json.load(open("math_102_topics.json"))
topics = [
    Topic(
        name=topic["name"],
        subtopics=[Subtopic(name=subtopic) for subtopic in topic["subtopics"]],
    )
    for topic in topics
]
topics[0].model_dump()

{'name': 'Set Theory',
 'subtopics': [{'name': 'Representations of Sets', 'concepts': []},
  {'name': 'Classifying Sets', 'concepts': []},
  {'name': 'Set Operations: Union and Intersection', 'concepts': []},
  {'name': 'Understanding Set Difference', 'concepts': []},
  {'name': 'Exploring Set Complement', 'concepts': []}]}

In [75]:
question = questions[1]
topic = "Mapping & Functions Overview"
subtopic = "Function Operations"
QuestionWithTopicSubtopic.model_validate(
    dict(question.model_dump(), topic=topic, subtopic=subtopic),
    context={"topics": topics},
).model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$',
 'topic': 'Mappin

In [76]:
topics_str = deindent(
    f"""
TOPICS:

{json.dumps([topic.model_dump() for topic in topics], indent=2)}
"""
)

topic_subtopic_message = deindent(
    f"""
You are a world class math course instructor.
You will be given a question with a 'problem' and a 'solution'.
Given these topics and subtopics below, assign a 'topic' and a 'subtopic' to the question.
The 'subtopic' must be one of the subtopics of the 'topic'.

{topics_str}
    """
)

subqs_message = deindent(
    f"""
You are a world class math course instructor.
You will be given a question with a 'problem', a 'solution', a 'topic', and a 'subtopic'.
Based on the main question's problem and solution, break the question down into 3-5 smaller subquestions.
Answering these questions in sequence should lead to the solution of the main question.
Make sure that the answer to the last subquestion is the solution to the main question.
So the subquestions are basically the steps to solve the main question.
And if a student can solve the main question, we can assume that they have learned the underlying concepts of the subquestions.
No 2 subquestions can have the same concept.
For each subquestion:
    1. Define the 'problem'.
    2. Give a detailed 'solution'.
    3. Given these topics below, assign a 'topic' and a 'subtopic' to the subquestion.
       The 'subtopic' must be one of the subtopics of the 'topic'.

{topics_str}
"""
)


def concepts_str(concepts: list[str]) -> str:
    return deindent(
        f"""
You can use a concept from the list below or come up with a new one if needed.

CONCEPTS:

{json.dumps(concepts, indent=2)}
"""
    )


def question_w_concept_message(concepts: list[str]) -> str:
    prompt = deindent(
        f"""
You are a world class math course instructor.
You will be given a question with a 'problem', a 'solution', a 'topic', and a 'subtopic'.
Assign a 'concept' to the question. The 'concept' should have at least {CONCEPT_WORD_COUNT} words.
Solving this question should help students understand this concept.
"""
    )
    if len(concepts) > 0:
        prompt += concepts_str(concepts)
    return prompt


def question_w_subqs_concept_message(concepts: list[str]) -> str:
    prompt = deindent(
        f"""
You are a world class math course instructor.
You will be given a question with a 'problem', a 'solution', a 'topic', and a 'subtopic'.
It will also have 3-5 subquestions. Each subquestion will have a 'problem', a 'solution', a 'topic', a 'subtopic', and a 'concept'.
Based on the subquestions, assign a 'concept' to the main question. The 'concept' should have at least {CONCEPT_WORD_COUNT} words.
Try not to repeat the concepts of the subquestions. Because the subquestions are the steps to solve the main question.
Solving this question should help students understand this concept.
"""
    )
    if len(concepts) > 0:
        prompt += concepts_str(concepts)
    return prompt

In [77]:
print(
    question_w_subqs_concept_message(
        ["addition", "subtraction", "multiplication", "division"]
    )
)

You are a world class math course instructor.
You will be given a question with a 'problem', a 'solution', a 'topic', and a 'subtopic'.
It will also have 3-5 subquestions. Each subquestion will have a 'problem', a 'solution', a 'topic', a 'subtopic', and a 'concept'.
Based on the subquestions, assign a 'concept' to the main question. The 'concept' should have at least 3 words.
Try not to repeat the concepts of the subquestions. Because the subquestions are the steps to solve the main question.
Solving this question should help students understand this concept.You can use a concept from the list below or come up with a new one if needed.

CONCEPTS:

[
  "addition",
  "subtraction",
  "multiplication",
  "division"
]


In [78]:
count_gpt_tokens(topic_subtopic_message), count_gpt_tokens(subqs_message)

(1869, 2012)

In [79]:
question = questions[1]
question_message = deindent(f"Question:\n\n{question.model_dump_json(indent=2)}")
print(question_message)

Question:

{
  "id": "6349",
  "problem": "Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$",
  "solution": "From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$"
}


In [80]:
question_topic_subtopic = ask_cld.create(
    system=topic_subtopic_message,
    messages=[
        user_message(content=question_message),  # type: ignore
    ],
    max_tokens=2048,
    model=MODEL,
    response_model=TopicSubtopic,
    max_retries=1,
    validation_context={"topics": topics},
)

In [81]:
question_topic_subtopic

TopicSubtopic(topic='Trigonometry Essentials', subtopic='Solving Trigonometric Identities')

In [82]:
question_w_topic_subtopic = QuestionWithTopicSubtopic.model_construct(
    **question.model_dump(), **question_topic_subtopic.model_dump()
)
question_w_topic_subtopic.model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$',
 'topic': 'Trigon

In [83]:
question_w_topic_subtopic_message = deindent(
    f"Question with Topic and Subtopic:\n\n{question_w_topic_subtopic.model_dump_json(indent=2)}"  # type: ignore
)
print(question_w_topic_subtopic_message)

Question with Topic and Subtopic:

{
  "id": "6349",
  "problem": "Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$",
  "solution": "From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\bo

In [84]:
subquestions_w_topic_subtopic = ask_cld.create(
    system=subqs_message,
    messages=[user_message(content=question_w_topic_subtopic_message)],  # type: ignore
    max_tokens=2048,
    model=MODEL,
    response_model=QuestionsWithTopicSubtopic,
    max_retries=1,
    validation_context={"topics": topics},
)

In [85]:
subquestions_w_topic_subtopic.model_dump()

{'questions': [{'id': '1',
   'problem': 'What is the relationship between the angles a and b given in the problem?',
   'solution': 'The problem states that $\\cos (a + b) = \\cos a + \\cos b$, which is a trigonometric identity. This means that the angles a and b are related in some way, and the solution will involve manipulating this identity to find the maximum value of $\\cos a$.',
   'topic': 'Trigonometry Essentials',
   'subtopic': 'Solving Trigonometric Identities'},
  {'id': '2',
   'problem': 'How can we express $\\cos a$ in terms of $\\cos (a + b)$ and $\\cos b$?',
   'solution': 'From the given identity $\\cos (a + b) = \\cos a + \\cos b$, we can rearrange to get $\\cos a = \\cos (a + b) - \\cos b$. This will be a key step in finding the maximum value of $\\cos a$.',
   'topic': 'Trigonometry Essentials',
   'subtopic': 'Solving Trigonometric Identities'},
  {'id': '3',
   'problem': 'How can we use the sum-to-product identity to further simplify the expression for $\\cos a

In [97]:
topics = json.load(open("math_102_topics.json"))
topics = [
    Topic(
        name=topic["name"],
        subtopics=[Subtopic(name=subtopic) for subtopic in topic["subtopics"]],
    )
    for topic in topics
]
topics[0].model_dump()

{'name': 'Set Theory',
 'subtopics': [{'name': 'Representations of Sets', 'concepts': []},
  {'name': 'Classifying Sets', 'concepts': []},
  {'name': 'Set Operations: Union and Intersection', 'concepts': []},
  {'name': 'Understanding Set Difference', 'concepts': []},
  {'name': 'Exploring Set Complement', 'concepts': []}]}

In [98]:
subquestions_concepts = []
for subquestion_idx, subquestion in enumerate(subquestions_w_topic_subtopic.questions):
    topic_id, topic = next(
        (i, topic) for i, topic in enumerate(topics) if topic.name == subquestion.topic
    )
    subtopic_id, subtopic = next(
        (i, subtopic)
        for i, subtopic in enumerate(topic.subtopics)
        if subtopic.name == subquestion.subtopic
    )
    print(f"{subtopic.model_dump_json(indent=2)}\n\n")
    concept_prompt = question_w_concept_message(concepts=subtopic.concepts)
    subquestion_message = deindent(
        f"Question:\n\n{subquestion.model_dump_json(indent=2)}"
    )
    try:
        for attempt in Retrying(
            wait=wait_random_exponential(min=30, max=60), stop=stop_after_attempt(3)
        ):
            with attempt:
                subquestion_concept = ask_cld.create(
                    system=concept_prompt,
                    messages=[user_message(content=subquestion_message)],  # type: ignore
                    max_tokens=2048,
                    model=MODEL,
                    response_model=Concept,  # type: ignore
                    max_retries=1,
                    validation_context={"topics": topics},
                ).concept
                subquestions_concepts.append(subquestion_concept)
                subtopic.concepts.append(subquestion_concept)
                subtopic.concepts = list(set(subtopic.concepts))
                topics[topic_id].subtopics[subtopic_id] = subtopic
    except RetryError as e:
        print(f"Failed to generate concept for subquestion: {subquestion_idx}: {e}")
        continue

{
  "name": "Solving Trigonometric Identities",
  "concepts": []
}


{
  "name": "Solving Trigonometric Identities",
  "concepts": [
    "Relationship between Complementary Angles in Trigonometric Identities"
  ]
}


{
  "name": "Solving Trigonometric Identities",
  "concepts": [
    "Relationship between Complementary Angles in Trigonometric Identities"
  ]
}


{
  "name": "Solving Trigonometric Identities",
  "concepts": [
    "Relationship between Complementary Angles in Trigonometric Identities",
    "Simplifying Trigonometric Expressions Using Sum-to-Product Identities"
  ]
}


{
  "name": "Solving Trigonometric Identities",
  "concepts": [
    "Relationship between Trigonometric Functions and Their Maximum Values",
    "Relationship between Complementary Angles in Trigonometric Identities",
    "Simplifying Trigonometric Expressions Using Sum-to-Product Identities"
  ]
}




In [100]:
subquestions_concepts

['Relationship between Complementary Angles in Trigonometric Identities',
 'Relationship between Complementary Angles in Trigonometric Identities',
 'Simplifying Trigonometric Expressions Using Sum-to-Product Identities',
 'Relationship between Trigonometric Functions and Their Maximum Values',
 'Relationship between Trigonometric Functions and Their Maximum Values']

In [101]:
question_w_topic_subtopic_and_concept_subquestions = (
    QuestionWithTopicSubtopicAndConceptSubquestions.model_construct(
        **question_w_topic_subtopic.model_dump(),
        subquestions=[
            QuestionWithConcept.model_construct(
                **subquestion.model_dump(), concept=concept
            )
            for subquestion, concept in zip(
                subquestions_w_topic_subtopic.questions, subquestions_concepts
            )
        ],
    )
)
question_w_topic_subtopic_and_concept_subquestions.model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$',
 'topic': 'Trigon

In [102]:
topic_id, topic = next(
    (i, topic)
    for i, topic in enumerate(topics)
    if topic.name == question_w_topic_subtopic_and_concept_subquestions.topic
)
subtopic_id, subtopic = next(
    (i, subtopic)
    for i, subtopic in enumerate(topic.subtopics)
    if subtopic.name == question_w_topic_subtopic_and_concept_subquestions.subtopic
)
subtopic.concepts = list(set(subtopic.concepts))
print(f"{subtopic.model_dump_json(indent=2)}\n\n")

{
  "name": "Solving Trigonometric Identities",
  "concepts": [
    "Relationship between Trigonometric Functions and Their Maximum Values",
    "Relationship between Complementary Angles in Trigonometric Identities",
    "Simplifying Trigonometric Expressions Using Sum-to-Product Identities"
  ]
}




In [103]:
question_concept = ask_cld.create(
    system=question_w_subqs_concept_message(concepts=subtopic.concepts),
    messages=[
        user_message(
            f"Question with Subquestions:\n\n{question_w_topic_subtopic_and_concept_subquestions.model_dump_json(indent=2)}"  # type: ignore
        )
    ],
    max_tokens=2048,
    model=MODEL,
    response_model=Concept,
    max_retries=1,
    validation_context={"topics": topics},
).concept

In [104]:
question_concept

'Relationship between Complementary Angles in Trigonometric Identities and Simplifying Trigonometric Expressions Using Sum-to-Product Identities'

In [105]:
subtopic.concepts.append(question_concept)
subtopic.concepts = list(set(subtopic.concepts))
topics[topic_id].subtopics[subtopic_id] = subtopic

In [108]:
question_w_concept_and_subquestions = (
    QuestionWithConceptAndSubquestions.model_construct(
        **question_w_topic_subtopic_and_concept_subquestions.model_dump(
            exclude={"subquestions"}
        ),
        concept=question_concept,
        subquestions=question_w_topic_subtopic_and_concept_subquestions.subquestions,
    )
)

In [109]:
question_w_concept_and_subquestions.model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$',
 'topic': 'Trigon

In [110]:
question_w_topic_subtopic_and_concept_subquestions.model_dump()

{'id': '6349',
 'problem': 'Let $a$ and $b$ be angles such that\n\\[\\cos (a + b) = \\cos a + \\cos b.\\]Find the maximum value of $\\cos a.$',
 'solution': 'From $\\cos (a + b) = \\cos a + \\cos b,$ $\\cos a = \\cos (a + b) - \\cos b.$  Then from sum-to-product,\n\\[\\cos (a + b) - \\cos b = -2 \\sin \\frac{a + 2b}{2} \\sin \\frac{a}{2}.\\]Let $k = \\sin \\frac{a + 2b}{2},$ so\n\\[\\cos a = -2k \\sin \\frac{a}{2}.\\]Then\n\\[\\cos^2 a = 4k^2 \\sin^2 \\frac{a}{2} = 4k^2 \\cdot \\frac{1}{2} (1 - \\cos a) = 2k^2 (1 - \\cos a),\\]so\n\\[\\frac{\\cos^2 a}{1 - \\cos a} = 2k^2 \\le 2.\\]Then $\\cos^2 a \\le 2 - 2 \\cos a,$ so\n\\[\\cos^2 a + 2 \\cos a + 1 \\le 3.\\]This means $(\\cos a + 1)^2 \\le 3,$ so $\\cos a + 1 \\le \\sqrt{3},$ or $\\cos a \\le \\sqrt{3} - 1.$\n\nEquality occurs if we take $a = \\arccos (\\sqrt{3} - 1)$ and $b = \\frac{3 \\pi - a}{2}$ (which will make $k = \\sin \\frac{a + 2b}{2} = -1$), so the maximum value of $\\cos a$ is $\\boxed{\\sqrt{3} - 1}.$',
 'topic': 'Trigon